# Clasificación de Texto Usando Embeddings Pre-entrenados

Este notebook realiza la clasificación de texto para identificar al autor de un texto entre tres autores posibles, utilizando embeddings pre-entrenados de GloVe y redes neuronales feed-forward (FFNN). Procesa los datos de texto, entrena múltiples arquitecturas de redes neuronales y evalúa su rendimiento en base a la precisión, exactitud y recall.

## 0. Importación de Librerias y Embeddings

In [1]:
import os
import requests
import zipfile

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import (
    utils,
    layers,
    models,
    callbacks
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
def download_glove(url: str, zip_path: str):
    """
    Descarga el archivo GloVe si no está presente localmente.
    
    Args:
        url (str): URL de descarga del archivo GloVe.
        zip_path (str): Ruta donde se almacenará el archivo descargado.
    """
    if not os.path.exists(zip_path):
        print(f"Descargando embeddings desde {url}...")
        response = requests.get(url)
        with open(zip_path, 'wb') as file:
            file.write(response.content)

def extract_glove(zip_path: str, glove_dir: str, glove_file: str):
    """
    Extrae los archivos GloVe del archivo zip si no están ya extraídos.
    
    Args:
        zip_path (str): Ruta del archivo zip de GloVe.
        glove_dir (str): Directorio de destino de los archivos extraídos.
        glove_file (str): Nombre del archivo GloVe específico a extraer.
    """
    if not os.path.exists(os.path.join(glove_dir, glove_file)):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(glove_file, glove_dir)

def load_embeddings(dim: int) -> dict[str, np.ndarray]:
    """
    Carga los embeddings preentrenados de GloVe y los devuelve en un diccionario.
    
    Args:
        dim (int): Dimensión de los embeddings a cargar.
    
    Returns:
        dict: Diccionario con palabras como claves y vectores como valores.
    """
    # Definir las rutas y nombres relevantes.
    glove_download_url = "https://nlp.stanford.edu/data/glove.6B.zip"
    glove_zip_path = "glove.6B.zip"
    glove_dir = "data/glove/"
    glove_file = f'glove.6B.{dim}d.txt'

    # Descargar y extraer GloVe si es necesario.
    os.makedirs(glove_dir, exist_ok=True)
    download_glove(glove_download_url, glove_zip_path)
    extract_glove(glove_zip_path, glove_dir, glove_file)

    # Cargar los embeddings en un diccionario.
    embeddings_index = {}
    with open(os.path.join(glove_dir, glove_file), 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]  # La primera palabra es la clave.
            coefs = np.asarray(values[1:], dtype='float32')    # Los valores restantes son el vector.
            embeddings_index[word] = coefs

    print(f"Embeddings de {dim} dimensiones cargados exitosamente.")
    return embeddings_index

# Cargar los embeddings de GloVe con 50, 100 y 300 dimensiones.
glove_50 = load_embeddings(50)
glove_100 = load_embeddings(100)
glove_300 = load_embeddings(300)

Embeddings de 50 dimensiones cargados exitosamente.
Embeddings de 100 dimensiones cargados exitosamente.
Embeddings de 300 dimensiones cargados exitosamente.


In [3]:
# Cargar el dataset
df = pd.read_csv('data/classifier/sentences.csv')
df.head()

,author,sentence
0,Jane Austen,The family of Dashwood had long been settled i...
1,Jane Austen,"The old gentleman died: his will was read, and..."
2,Jane Austen,"No sooner was his father’s funeral over, than ..."
3,Jane Austen,"“Certainly not; but if you observe, people alw..."
4,Jane Austen,Edward Ferrars was not recommended to their go...


In [4]:
# Contar el número de datos por autor
author_counts = df['author'].value_counts()

# Crear DataFrame resumen
summary_df = author_counts.reset_index()
summary_df.columns = ['author', 'num_training_data']

summary_df

,author,num_training_data
0,Leo Tolstoy,866
1,Jane Austen,426
2,James Joyce,321


## 1. Preprocesamiento de los Datos

In [5]:
# Dividir en conjunto de entrenamiento, validación y prueba
x_train, x_temp, y_train, y_temp = train_test_split(df['sentence'], df['author'], train_size=0.7, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size=0.5, random_state=42)

# Tokenización usando Keras
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

# Convertir el texto en secuencias de enteros
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_val_seq = tokenizer.texts_to_sequences(x_val)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Rellenar las secuencias para que tengan la misma longitud
max_length = max([len(seq) for seq in x_train_seq])
x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post')
x_val_pad = pad_sequences(x_val_seq, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post')

## 2. Definición de los Modelos de Redes Neuronales

In [6]:
def create_glove_embedding_layer(embeddings, tokenizer, max_length):
    """
    Crea una capa de embeddings a partir de los embeddings GloVe.
    
    Args:
        embeddings (dict): Diccionario con los embeddings de GloVe.
        tokenizer (Tokenizer): Tokenizador de Keras con el índice de palabras.
        max_length (int): Longitud máxima de las secuencias de entrada.
    
    Returns:
        Embedding: Capa de embeddings de Keras con pesos pre-entrenados.
    """
    # Inicializar la matriz de embeddings con ceros
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, len(next(iter(embeddings.values())))))

    # Llenar la matriz de embeddings con los vectores de GloVe
    for word, i in tokenizer.word_index.items():
        vector = embeddings.get(word)
        if vector is not None:
            embedding_matrix[i] = vector

    # Crear y devolver la capa de embeddings
    return layers.Embedding(
        input_dim=len(tokenizer.word_index) + 1,
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        input_length=max_length,
        trainable=False
    )

# Crear capas de embeddings con GloVe
embedding_layer_50 = create_glove_embedding_layer(glove_50, tokenizer, max_length)
embedding_layer_100 = create_glove_embedding_layer(glove_100, tokenizer, max_length)
embedding_layer_300 = create_glove_embedding_layer(glove_300, tokenizer, max_length)

In [7]:
# Arquitectura 1: Modelo sencillo
def create_ffnn_model_1(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward simple.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = models.Sequential([
            layers.Input(shape=(max_length,)),
            embedding_layer,
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(3, activation='softmax')
        ])
    return model

# Arquitectura 2: Modelo con más capas
def create_ffnn_model_2(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward con más capas.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = models.Sequential([
            layers.Input(shape=(max_length,)),
            embedding_layer,
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(3, activation='softmax')
        ])
    return model

# Arquitectura 3: Modelo con más unidades
def create_ffnn_model_3(embedding_layer):
    """
    Crea un modelo de red neuronal feedforward con más unidades.

    Args:
    embedding_layer: Capa de embeddings de Keras utilizada como entrada.

    Returns:
    Sequential: Modelo de red neuronal compilado.
    """
    model = models.Sequential([
            layers.Input(shape=(max_length,)),
            embedding_layer,
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(3, activation='softmax')
        ])
    return model

## 3. Entrenamiento y Evaluación de los Modelos de Redes Neuronales

In [8]:
# Codificación de etiquetas (autores)
label_encoder = LabelEncoder()
y_train_encoded = utils.to_categorical(label_encoder.fit_transform(y_train))
y_val_encoded = utils.to_categorical(label_encoder.transform(y_val))
y_test_encoded = utils.to_categorical(label_encoder.transform(y_test))

In [9]:
def evaluate_model(model, x_test_pad, y_test_encoded):
    """
    Evalúa el rendimiento de un modelo entrenado calculando accuracy, precision y recall.
    
    Args:
    model (keras.models.Model): El modelo entrenado.
    x_test_pad (numpy.ndarray): Conjunto de datos de prueba preprocesados y tokenizados.
    y_test_encoded (numpy.ndarray): Etiquetas de prueba codificadas en formato one-hot.
    
    Returns:
    tuple: Un tupla que contiene:
        - accuracy (float): La proporción de predicciones correctas.
        - precision (float): La proporción de predicciones positivas correctas (precisión macro).
        - recall (float): La proporción de verdaderos positivos detectados (recall macro).
    """
    # Obtener predicciones del modelo
    y_pred = model.predict(x_test_pad)
    
    # Convertir las predicciones y etiquetas de one-hot a clases
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test_encoded, axis=1)
    
    # Calcular accuracy
    accuracy = np.mean(y_pred_classes == y_test_classes)
    
    # Calcular precisión y recall usando la métrica macro (promedio entre todas las clases)
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')
    
    return accuracy, precision, recall

In [10]:
# Iterar sobre las capas de embeddings y las dimensiones de los modelos Word2Vec (50, 100, 300 dimensiones)
for embedding_layer, dimensions in [(embedding_layer_50, 50), (embedding_layer_100, 100), (embedding_layer_300, 300)]:
    
    # Iterar sobre las funciones de creación de modelos FFNN (modelos 1, 2 y 3)
    for i, model_fn in enumerate([create_ffnn_model_1, create_ffnn_model_2, create_ffnn_model_3], 1):
        
        # Crear el modelo usando la capa de embeddings actual
        print(f"\nEntrenando Modelo {i} con {dimensions} dimensiones..." "\n")
        model = model_fn(embedding_layer)
        
        # Mostrar el resumen del modelo (capas y dimensiones)
        model.summary()

        # Compilar y entrenar el modelo
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        history = model.fit(x_train_pad, y_train_encoded, 
                            epochs=10, batch_size=32, 
                            validation_data=(x_val_pad, y_val_encoded), 
                            verbose=1)
        
        # Evaluar el modelo en el conjunto de prueba
        accuracy, precision, recall = evaluate_model(model, x_test_pad, y_test_encoded)

        # Mostrar los resultados finales de la evaluación (accuracy, precision y recall)
        print(f"\nEvaluación del Modelo {i} con embeddings de {dimensions} dimensiones - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}", "\n")


Entrenando Modelo 1 con 50 dimensiones...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 254, 50)           851050    
                                                                 
 flatten (Flatten)           (None, 12700)             0         
                                                                 
 dense (Dense)               (None, 128)               1625728   
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 2,477,165
Trainable params: 1,626,115
Non-trainable params: 851,050
_________________________________________________________________
Epoch 1/10
36/36 [==============================] - 2s 18ms/step - loss: 1.0675 - accuracy: 0.5173 - val_loss: 0.8250 - val_accu